In [13]:
import openai
import os
def get_openai_key():
    # Check if the file exists
    if not os.path.isfile('openai_key.txt'):
        # Create the file if it does not exist and write a default value or leave it blank
        with open('openai_key.txt', 'w') as file:
            file.write('')  # You could prompt the user for a key or leave it blank
    # Read the key from the file
    with open('openai_key.txt', 'r') as file:
        return file.read().strip()


# Usage
openai.api_key = get_openai_key()

In [16]:
detailed_description = 'a tutor for solving three rational and one square root equation'
# Introduction to the task
prompt_system = "Create a series of high-level steps for solving a problem in a tutor interface. Each step should be a clear, concise description of an action the user needs to take. The steps should logically progress from the initial problem statement to the final solution, adhering to pedagogical best practices."
prompt_introduction = """Generate a series of high-level steps for solving a problem in a tutor interface based on a detailed description."""
prompt_format = "The steps should be numbered and separated by commas. Each step should be a short, actionable phrase that guides the user through the problem-solving process."
prompt_design_instructions = """Design principles require that each step be focused on a single action or concept. Steps should be ordered logically and build upon each other. Avoid using technical jargon or complex language. Each step should be clear and easily understandable for the user."""
# Instruction to transform the description into the steps
prompt_task = """Transform the detailed description into a series of high-level steps according to the instructions."""
prompt_examples = "Example 0: A fraction is always of the form column{input[Numerator], label[____], input[Denominator]} and is never row{input[Numerator], label[/], input[Denominator]}. Steps: 1. Enter the numerator and denominator"
prompt_examples += "Example 1: Equation row with two fraction members should be represented as a row with multiple columns inside for the members. Each column contains stacked numerator, operand (division symbol), and denominator. Include labels for operators in the main row. For a fraction equation like 1/2 + 3/4, the layout should be row{column{input[Numerator], label[____], input[Denominator]}, label[+], column{input[Numerator], label[____], input[Denominator]}, label[=], input[Result]}. This format ensures that each part of the fraction equation is clearly defined and separated for input. Steps: 1. Enter the numerators and denominators of the fractions, 2. Add the fractions and enter the result \
"
# Examples of the prompt format
prompt_examples += ", Example 2: A two equations solver should be represented as title[2 Equation Solver], column{label[First Equation], row{input[First Equation Coefficient 1], label[x], label[+], input[First Equation Coefficient 2], label[=], input[First Equation Result]}, label[Second Equation], row{input[Second Equation Coefficient 1], label[x], label[+], input[Second Equation Coefficient 2], label[y], label[=], input[Second Equation Result]}}. Steps: 1. Enter the coefficients and variables of the first equation, 2. Enter the coefficients and variables of the second equation, 3. Solve the system of equations"
prompt_examples += ", Example 3: For a radicals tutor interface, the representation would be title[Radicals Tutor], column{label[Solve the following radicals multiplication problem below], row{label[What is √2 * √2], input[Result]}, input{label[Simplify the expression]}}. Steps: 1. Multiply the radicals, 2. Simplify the expression"
prompt_examples += ", Example 4: For a Squared Tutor interface, the representation would be title[Squared Tutor], column{label[Enter the number you wish to square], row{input[Number], label[=], input[Result]}}. This configuration places the label and input for the number and the result all in a single horizontal row, maintaining a clear and concise layout. Steps: 1. Enter the number to be squared, 2. Calculate the square of the number"
prompt_examples += ", Example 5: For a Tutor for missionaries and cannibals, the representation would be title[Missionaries and Cannibals Tutor], column{label[Instructions], row{label[Enter the number of missionaries], input[Missionaries]}, row{label[Enter the number of cannibals], input[Cannibals]}, label[Solution], row{input[First Move], label[->], input[Second Move]}, row{input[Third Move], label[->], input[Fourth Move]}}. As in This case, make sure to scaffold all the resolution steps. Steps: 1. Enter the number of missionaries and cannibals, 2. Determine the first move, 3. Determine the second move, 4. Determine the third move, 5. Determine the fourth move"
prompt_examples += ", Example 6: For a tutor for calculating proper drug dosage levels for a nurse to administer to a patient, the representation would be title[Drug Dosage Tutor], column{label[Calculation Instructions], row{label[Enter the weight of the patient (in kg)], input[Patient Weight]}, row{label[Enter the recommended dosage per kg (in mg)], input[Dosage per kg]}, label[Calculated Dosage], row{input[Dosage Total], label[mg]}}. Steps: 1. Enter the patient's weight and recommended dosage per kg, 2. Calculate the total dosage"
prompt_examples += ", Example 7: For A tutor for optimizing a business workflow, the representation would be title[Business Workflow Optimization Tutor], column{label[Instructions], row{label[Enter the number of employees], input[Number of Employees]}, row{label[Enter the number of hours worked by each employee per week], input[Hours Worked]}, row{label[Enter the average number of tasks completed by an employee per hour], input[Average Tasks Completed]}, label[Optimization Solution], row{input[Current Workflow Efficiency], label[%]}, row{label[Enter the changes you wish to make], input[Suggested Changes]}, label[Calculated Optimized Efficiency], row{input[Optimized Workflow Efficiency], label[%]}}. Steps: 1. Enter the number of employees, hours worked, and average tasks completed, 2. Calculate the current workflow efficiency, 3. Enter the suggested changes to optimize the workflow, 4. Calculate the optimized workflow efficiency"
prompt_examples += ", Example 8: For an English article selection tutor, the representation would be title[English Article Selection Tutor], column{label[Instructions], label[Select the most appropriate article ('a', 'an', 'the' or 'no article') for each blank in the sentences below], row{label[Sentence 1], input[Sentence 1 Article Choice]}, row{label[Sentence 2], input[Sentence 2 Article Choice]}, row{label[Sentence 3], input[Sentence 3 Article Choice]}}. Steps: 1. Select the appropriate article for the first sentence, 2. Select the appropriate article for the second sentence, 3. Select the appropriate article for the third sentence"
prompt_examples += ", Example 9: For a tutor for conducting a cash flow analysis of a business, the representation would be title[Cash Flow Analysis Tutor], column{label[Instructions], label[Enter the relevant business data in the fields below to begin your cash flow analysis conversion], row{label[Enter the total revenue of the business for the chosen period], input[Total Revenue]}, row{label[Enter the total cost of goods sold (COGS) during the same period], input[Cost of Goods Sold]}, label[Gross Profit], row{input[Gross Profit], label[]}, row{label[Enter the total operating expenses during the same period], input[Operating Expenses]}, label[Net Profit], row{input[Net Profit], label[]}, row{label[Enter the total cash flows from investing activities (e.g. purchase of assets, investments)], input[Investing Cash Flows]}, row{label[Enter the total cash flows from financing activities (e.g. loans, dividends, repayable)], input[Financing Cash Flows]}, label[Net Cash Flow], row{input[Net Cash Flow], label[]}}. Steps: 1. Enter the total revenue and cost of goods sold, 2. Calculate the gross profit, 3. Enter the total operating expenses, 4. Calculate the net profit, 5. Enter the cash flows from investing activities, 6. Enter the cash flows from financing activities, 7. Calculate the net cash flow"
prompt_examples += ", Example 10: For a tutor for a three members rational equation, the representation would be title[Rational Equations], column{label[Solve the rational equation], row{column{input[Numerator 1], label[___], input[Denominator 1]}, label[+], column{input[Numerator 2], label[___], input[Denominator 2]}, label[=], column{input[Numerator 3], label[___], input[Denominator 3]}}}. Steps: 1. Enter the numerators and denominators of the three fractions, 2. Solve the rational equation"


# Combining all the parts to form the complete prompt
prompt = f"{prompt_system}\n{prompt_introduction}\n\n{prompt_format}\n\n{prompt_design_instructions}\n\n{prompt_examples}\n\n{prompt_task}"
instruction = f"\nDetailed Description:\n{detailed_description}\n"

try:
    response = openai.chat.completions.create(
      model="gpt-4", # Adjusted to use GPT-4
    messages=[{"role": "system", "content": prompt}, {"role": "user", "content": instruction }]
    )
    generated_text = response.choices[0].message.content.strip()
    print(generated_text)
    # Return the high-level steps generated by OpenAI
    return jsonify({"steps": generated_text})
except Exception as e:
    return jsonify({"error": str(e)}), 500

Steps: 1. Enter the numerators and denominators for the three rational equations, 2. Solve the three rational equations, 3. Enter the value under the square root, 4. Solve the square root equation.


In [17]:
print(res)

{'steps': 'Steps: 1. Enter the numerators and denominators for the three rational equations, 2. Solve the three rational equations, 3. Enter the value under the square root, 4. Solve the square root equation.'}
